The wrong and Right Way to Do Cross-validation

This one is short, and no stretch goals available. In the interst of time, I also won't code review this assignment.

Re-implement the example in section 7.10.2 using any simple, out of the box classifier (like K nearest neighbors from sci-kit). Reproduce the results for the incorrect and correct way of doing cross-validation.

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest

N = 50 # 50 samples
p = 5000 # 5000 predictors

The wrong way

In [ ]:
acc = []

for i in range(50): # run simulation 50 times
  x = np.random.normal(size=(N, p)) # generate data
  x = (x - np.mean(x)) / np.std(x) # normalize data

  y = np.append(np.ones(25), np.zeros(25)) # generate labels with 50/50 split
  np.random.shuffle(y)

  # select 100 best predictors. the selection in the wrong solution selects based on all of the data, including the validation fold
  x_selected = SelectKBest(k=100).fit_transform(x, y)
  n_groups = 5

  x_folds = np.split(x_selected, n_groups) # split into n groups
  y_folds = np.split(y, n_groups)

  correct_guesses = 0

  # for every fold, remove it, and then classify based on the remaining folds.
  for i in range(n_groups):
    x_removed = x_folds.copy()
    x_removed.pop(i)
    x_removed = np.concatenate(np.array(x_removed))

    y_removed = y_folds.copy()
    y_removed.pop(i)
    y_removed = np.concatenate(np.array(y_removed))

    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(x_removed, y_removed)

    # make predictions based on removed fold
    yh = knn.predict(x_folds[i])
    guesses = np.diff([yh, y_folds[i]], axis=0)
    correct_guesses += guesses.size - np.count_nonzero(guesses)
  acc.append(100.0 * (correct_guesses/ N))

# the incorrect method yields about 2.5% error, which is similar to the textbook and significantly lower than the true error 
print("Incorrect Method accuracy: " + str(np.mean(acc)) + "%")

Incorrect Method accuracy: 98.68%


The right way

In [ ]:
acc = []

for i in range(50):
  x = np.random.normal(size=(N, p))
  x = (x - np.mean(x)) / np.std(x)

  y = np.append(np.ones(25), np.zeros(25))
  np.random.shuffle(y)

  n_groups = 5

  x_folds = np.split(x_selected, n_groups)
  y_folds = np.split(y, n_groups)

  correct_guesses = 0

  for i in range(n_groups):
    x_removed = x_folds.copy()
    x_removed.pop(i)
    x_removed = np.concatenate(np.array(x_removed))

    y_removed = y_folds.copy()
    y_removed.pop(i)
    y_removed = np.concatenate(np.array(y_removed))

    # this time, the k best predictors were selected after the validation fold was removed.
    x_removed = SelectKBest(k=100).fit_transform(x_removed, y_removed)

    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(x_removed, y_removed)

    yh = knn.predict(x_folds[i])
    guesses = np.diff([yh, y_folds[i]], axis=0)
    correct_guesses += guesses.size - np.count_nonzero(guesses)
  acc.append(100.0 * (correct_guesses/ N))

# Correct method accuracy is about the true error rate at 50%
print("Correct Method accuracy: " + str(np.mean(acc)) + "%")

Correct Method accuracy: 48.28%
